<a href="https://colab.research.google.com/github/Suf1an65/Final-Year-Project/blob/main/notebooks/Final_Scraping_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version


Python 3.12.12


In [106]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [107]:
!pip install yt-dlp

In [61]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [62]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 3,917 B in 2s (2,500 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [63]:
!apt-get install -y nodejs npm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
npm is already the newest version (8.5.1~ds-1).
nodejs is already the newest version (12.22.9~dfsg-1ubuntu3.6).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [64]:
!node --version  # Verify installation

v12.22.9


In [65]:
!pip show yt-dlp

Name: yt-dlp
Version: 2025.11.12
Summary: A feature-rich command-line audio/video downloader
Home-page: 
Author: 
Author-email: pukkandan <pukkandan.ytdlp@gmail.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: 


In [73]:
# Install ffmpeg in Colab
!apt-get update -qq
!apt-get install -y ffmpeg

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [2]:
import pandas as pd
import numpy as np
import yt_dlp
import os
import glob
import time

# Install ffmpeg in Colab
!apt-get update -qq
!apt-get install -y ffmpeg

# -----------------------------
# PATHS
# -----------------------------
base_dir = '/content/drive/MyDrive/Final-Year-Project'
video_directory = os.path.join(base_dir, 'data/WebScrapedVideos/')
excel_path = os.path.join(base_dir, 'data/spreadsheets/Dolos.xlsx')
cookie_path = os.path.join(base_dir, 'Notebooks/cookies.txt')
output_csv_path = os.path.join(base_dir, 'data/spreadsheets/my_dataset_with_paths.csv')

os.makedirs(video_directory, exist_ok=True)

# -----------------------------
# BATCH CONFIGURATION
# -----------------------------
START_BUFFER = 0.0      # No buffer for face consistency
END_BUFFER = 0.0        # No buffer to avoid pan-out
BATCH_SIZE = 1         # Process 50 at a time
START_INDEX = 19       # Change this to resume: 0, 50, 100, 150, etc.

# -----------------------------
# LOAD DATAFRAME
# -----------------------------
df = pd.read_excel(excel_path, skiprows=2)

# Drop extra columns (keep only first 8)
columns_to_drop = [num for num in range(8, len(df.columns))]
df.drop(df.columns[columns_to_drop], axis=1, inplace=True)

# Add tracking columns
if 'video_path' not in df.columns:
    df['video_path'] = ""
if 'status' not in df.columns:
    df['status'] = ""

print("=== DATASET INFO ===")
print(f"Total clips in dataset: {len(df)}")
print(f"Columns: {df.columns.tolist()}")

# -----------------------------
# TIMESTAMP FUNCTIONS
# -----------------------------
def fix_timestamp(t):
    if pd.isna(t):
        return None
    t = str(t).strip()
    parts = t.split(':')

    if len(parts) == 3:
        minutes = parts[0]
        seconds = parts[1]
        return f"00:{minutes.zfill(2)}:{seconds.zfill(2)}"
    elif len(parts) == 2:

      return f"00:{parts[0].zfill(2)}:{parts[1].zfill(2)}"
    else:
        return None

def timestamp_to_seconds(timestamp):
    if not timestamp:
        return None
    parts = timestamp.split(':')

    hours = int(parts[0])

    minutes = int(parts[1])


    seconds = int(parts[2])
    return hours * 3600 + minutes * 60 + seconds

# Apply timestamp fixes
df['Start time in the episode'] = df['Start time in the episode'].apply(fix_timestamp)
df['End time in the episode'] = df['End time in the episode'].apply(fix_timestamp)

# -----------------------------
# CALCULATE BATCH RANGE
# -----------------------------
start_idx = START_INDEX
end_idx = min(START_INDEX + BATCH_SIZE, len(df))

print(f"\n=== BATCH PROCESSING ===")
print(f"Processing rows: {start_idx} to {end_idx-1}")
print(f"Batch size: {end_idx - start_idx}")
print(f"Start buffer: {START_BUFFER}s | End buffer: {END_BUFFER}s")
print(f"Strategy: Face consistency priority (no buffer)\n")

# Check how many already done
already_done = df.iloc[start_idx:end_idx]['status'].eq("Success").sum()
if already_done > 0:
    print(f"⚠️  {already_done} clips already downloaded in this batch")
    print(f"Will skip these and process remaining {end_idx - start_idx - already_done}\n")

# -----------------------------
# DOWNLOAD LOOP
# -----------------------------
failure_count = 0
success_count = 0
start_time = time.time()

for index, row in df.iloc[start_idx:end_idx].iterrows():
    # Skip if already successful
    if df.at[index, 'status'] == "Success":
        success_count += 1
        print(f"Row {index}: Already downloaded ✓")
        continue

    video_url = row['Video URL']
    start_time_str = row['Start time in the episode']
    end_time_str = row['End time in the episode']

    start_seconds = timestamp_to_seconds(start_time_str)
    end_seconds = timestamp_to_seconds(end_time_str)

    # Apply buffer
    buffer_start = max(0, start_seconds - START_BUFFER)
    buffer_end = end_seconds + END_BUFFER

    clip_num = success_count + failure_count + 1 - already_done
    total_to_process = end_idx - start_idx - already_done

    print(f"\n=== Row {index} ({clip_num}/{total_to_process}) ===")
    print(f"Time: {start_time_str} to {end_time_str} ({end_seconds - start_seconds}s)")

    file_name = f"{index:04d}"
    full_output_path = os.path.join(video_directory, file_name)

    try:
        ydl_opts = {
            'format': 'best[height<=720]',
            'merge_output_format': 'mp4',
            'download_ranges': yt_dlp.utils.download_range_func(None, [(buffer_start, buffer_end)]),
            'force_keyframes_at_cuts': True,
            'noplaylist': True,
            'postprocessor_args': ['-c', 'copy'],
            'outtmpl': full_output_path,
            'cookiefile': cookie_path,
            'quiet': True,
            'no_warnings': True,
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])

        # Find created file (handles any extension)
        possible_files = glob.glob(f"{full_output_path}*")

        if possible_files:
            output_file = possible_files[0]
            file_size = os.path.getsize(output_file) / 1024

            if file_size > 10:
                df.at[index, 'video_path'] = output_file
                df.at[index, 'status'] = "Success"
                success_count += 1
                print(f"✓ {os.path.basename(output_file)} ({file_size:.1f} KB)")
            else:
                raise Exception(f"File too small: {file_size:.1f} KB")
        else:
            raise Exception("No output file found")

    except Exception as e:
        print(f"✗ Failed: {e}")
        df.at[index, 'video_path'] = "DOWNLOAD_FAILED"
        df.at[index, 'status'] = str(e)
        failure_count += 1

    # Save progress after each download
    df.to_csv(output_csv_path, index=False)

    # Rate limiting
    time.sleep(0.5)

# -----------------------------
# BATCH SUMMARY
# -----------------------------
elapsed_time = time.time() - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"\n{'='*50}")
print(f"BATCH {START_INDEX}-{end_idx-1} COMPLETE")
print(f"{'='*50}")
print(f"✓ Success: {success_count}")
print(f"✗ Failed: {failure_count}")
print(f"⏱️  Time: {minutes}m {seconds}s")

if success_count + failure_count > 0:
    success_rate = success_count / (success_count + failure_count) * 100
    print(f"📊 Success rate: {success_rate:.1f}%")

    # Estimate remaining time
    if success_count + failure_count - already_done > 0:
        avg_time_per_clip = elapsed_time / (success_count + failure_count - already_done)
        remaining_clips = len(df) - end_idx
        remaining_batches = (remaining_clips + BATCH_SIZE - 1) // BATCH_SIZE
        estimated_minutes = (remaining_clips * avg_time_per_clip) / 60
        print(f"⏳ Estimated time for remaining {remaining_clips} clips: ~{estimated_minutes:.0f}m ({remaining_batches} batches)")

# Show dataset progress
total_success = df['status'].eq("Success").sum()
total_failed = df['status'].eq("DOWNLOAD_FAILED").sum()
print(f"\n📁 OVERALL DATASET PROGRESS")
print(f"Total downloaded: {total_success}/{len(df)} ({total_success/len(df)*100:.1f}%)")
print(f"Total failed: {total_failed}")

# Show next steps
if end_idx < len(df):
    print(f"\n🔄 TO CONTINUE:")
    print(f"Change START_INDEX to {end_idx}")
    print(f"Then run this cell again")
else:
    print(f"\n🎉 ALL CLIPS PROCESSED!")
    print(f"Check your CSV: {output_csv_path}")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
=== DATASET INFO ===
Total clips in dataset: 1678
Columns: ['Video URL', 'File name of the video clip ', 'Label "truth" or "deception"', 'Start time in the episode', 'End time in the episode', 'Name of the game show', 'Participants name', 'Participants gender', 'video_path', 'status']
This is the fixed timestamp it returns:  00:02:06
This is the fixed timestamp it returns:  00:02:25
This is the fixed timestamp it returns:  00:05:50
This is the fixed timestamp it returns:  00:07:03
This is the fixed timestamp it returns:  00:07:32
This is the fixed timestamp it returns:  00:08:58

TypeError: can only concatenate str (not "int") to str

## I'm going to break this down into blocks
### First block is import statements and packages

In [88]:
import pandas as pd
import numpy as np
import yt_dlp
import os
import glob
import time

# Install ffmpeg in Colab
!apt-get update -qq
!apt-get install -y ffmpeg


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


### Setting up the paths for directories

In [89]:
# -----------------------------
# PATHS
# -----------------------------
base_dir = '/content/drive/MyDrive/Final-Year-Project'
video_directory = os.path.join(base_dir, 'data/WebScrapedVideos/')
excel_path = os.path.join(base_dir, 'data/spreadsheets/Dolos.xlsx')
cookie_path = os.path.join(base_dir, 'Notebooks/cookies.txt')
output_csv_path = os.path.join(base_dir, 'data/spreadsheets/my_dataset_with_paths.csv')

os.makedirs(video_directory, exist_ok=True)

### Loading the data frame and batch configurations

In [90]:
# -----------------------------
# BATCH CONFIGURATION
# -----------------------------
START_BUFFER = 0.0      # No buffer for face consistency
END_BUFFER = 0.0        # No buffer to avoid pan-out
BATCH_SIZE = 1        # Process 50 at a time
START_INDEX = 20        # Change this to resume: 0, 50, 100, 150, etc.

# -----------------------------
# LOAD DATAFRAME
# -----------------------------
# Force time columns to be read as strings to avoid Excel datetime conversion
df = pd.read_excel(
    excel_path,
    skiprows=2,
    dtype={
        'Start time in the episode': str,
        'End time in the episode': str
    }
)

# Drop extra columns (keep only first 8)
columns_to_drop = [num for num in range(8, len(df.columns))]
df.drop(df.columns[columns_to_drop], axis=1, inplace=True)

# Add tracking columns
if 'video_path' not in df.columns:
    df['video_path'] = ""
if 'status' not in df.columns:
    df['status'] = ""

print("=== DATASET INFO ===")
print(f"Total clips in dataset: {len(df)}")
print(f"Columns: {df.columns.tolist()}")

# Check raw timestamps before conversion
print("\n=== RAW TIMESTAMPS (First 5 + Row 19) ===")
for i in [0, 1, 2, 3, 4, 19]:
    if i < len(df):
        start = df.iloc[i]['Start time in the episode']
        end = df.iloc[i]['End time in the episode']
        print(f"Row {i}: Start='{start}' | End='{end}'")


=== DATASET INFO ===
Total clips in dataset: 1678
Columns: ['Video URL', 'File name of the video clip ', 'Label "truth" or "deception"', 'Start time in the episode', 'End time in the episode', 'Name of the game show', 'Participants name', 'Participants gender', 'video_path', 'status']

=== RAW TIMESTAMPS (First 5 + Row 19) ===
Row 0: Start='02:06:00' | End='02:11:00'
Row 1: Start='02:25:00' | End='02:32:00'
Row 2: Start='05:50:00' | End='05:55:00'
Row 3: Start='07:03:00' | End='07:06:00'
Row 4: Start='07:32:00' | End='07:36:00'
Row 19: Start='1 day, 1:26:00' | End='1 day, 1:31:00'


### TimeStamp functions

In [104]:
def fix_timestamp(t):
    """
    Convert various timestamp formats to HH:MM:SS format
    Handles cases where minutes exceed 60 or duration exceeds 24 hours
    """
    if pd.isna(t):
        return None

    t = str(t).strip()

    # --- Handle "X day(s), HH:MM:SS" format ---
    if 'day' in t.lower():
        # Keep your existing logic for day formatting - this handles long videos
        # ... (Your existing logic here) ...
        parts = t.split(',')
        time_part = parts[1].strip() if len(parts) > 1 else "00:00:00"

        # Parse the time part
        time_components = time_part.split(':')
        hours = int(time_components[0]) if len(time_components) > 0 else 0
        minutes = int(time_components[1]) if len(time_components) > 1 else 0
        seconds = int(time_components[2]) if len(time_components) > 2 else 0

        # Add 24 hours per day (assuming only one day for simplicity based on your original logic)
        total_hours = 24 + hours
        total_minutes = total_hours * 60 + minutes
        total_seconds = total_minutes * 60 + seconds

        # Convert back to HH:MM:SS
        final_hours = total_seconds // 3600
        final_minutes = (total_seconds % 3600) // 60
        final_seconds = total_seconds % 60

        return f"{final_hours:02d}:{final_minutes:02d}:{final_seconds:02d}"

    # --- Handle normal format (H:M:S, M:S:MS, or M:S) ---
    parts = t.split(':')

    if len(parts) == 3:
        # If the user input '25:35:00' should mean 25 minutes and 35 seconds,
        # then the input is M:S:MS (Minutes:Seconds:Milliseconds/Frames)
        minutes = int(parts[0])
        seconds = int(parts[1])
        # Ignore the third part (milliseconds/frames)

        total_seconds = minutes * 60 + seconds

        # Output directly as HH:MM:SS
        final_hours = total_seconds // 3600
        final_minutes = (total_seconds % 3600) // 60
        final_seconds = total_seconds % 60

        # For '25:35:00' (M:S:MS), this will return '00:25:35' (1535 seconds)
        return f"{final_hours:02d}:{final_minutes:02d}:{final_seconds:02d}"

    elif len(parts) == 2:
        # Assume MM:SS format and convert to HH:MM:SS
        minutes = int(parts[0])
        seconds = int(parts[1])

        total_seconds = minutes * 60 + seconds
        final_hours = total_seconds // 3600
        final_minutes = (total_seconds % 3600) // 60
        final_seconds = total_seconds % 60

        return f"{final_hours:02d}:{final_minutes:02d}:{final_seconds:02d}"

    else:
        return None
def timestamp_to_seconds(t):
    """
    Converts timestamp strings into total seconds.
    Handles formats:
    - 'MM:SS'
    - 'HH:MM:SS'
    - '1 day, HH:MM:SS'
    - '2 days, HH:MM:SS'
    - Also handles Excel weird formats.
    """
    if pd.isna(t):
        return None

    t = str(t).strip()

    # --- Case 1: Includes "day" ---
    if "day" in t.lower():
        # Example: "1 day, 1:35:00"
        parts = t.split(",")
        day_part = parts[0].strip()
        time_part = parts[1].strip()

        # Extract number of days
        days = int(day_part.split()[0])

        # Extract H:M:S
        h, m, s = time_part.split(":")
        hours = int(h)
        minutes = int(m)
        seconds = int(s)

        return days * 86400 + hours * 3600 + minutes * 60 + seconds

    # --- Case 2: HH:MM:SS ---
    parts = t.split(":")
    if len(parts) == 3:
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        return hours * 3600 + minutes * 60 + seconds

    # --- Case 3: MM:SS ---
    if len(parts) == 2:
        minutes = int(parts[0])
        seconds = int(parts[1])
        return minutes * 60 + seconds

    # Unknown format
    return None
start_time_str = fix_timestamp(row['Start time in the episode'])
end_time_str = fix_timestamp(row['End time in the episode'])

Main Code for downloading and showing progress

In [105]:
# -----------------------------
# CALCULATE BATCH RANGE
# -----------------------------
start_idx = START_INDEX
end_idx = min(START_INDEX + BATCH_SIZE, len(df))

print(f"\n=== BATCH PROCESSING ===")
print(f"Processing rows: {start_idx} to {end_idx-1}")
print(f"Batch size: {end_idx - start_idx}")
print(f"Start buffer: {START_BUFFER}s | End buffer: {END_BUFFER}s")
print(f"Strategy: Face consistency priority (no buffer)\n")

# Check how many already done
already_done = df.iloc[start_idx:end_idx]['status'].eq("Success").sum()
if already_done > 0:
    print(f"⚠️  {already_done} clips already downloaded in this batch")
    print(f"Will skip these and process remaining {end_idx - start_idx - already_done}\n")

# -----------------------------
# DOWNLOAD LOOP
# -----------------------------
failure_count = 0
success_count = 0
start_time = time.time()

for index, row in df.iloc[start_idx:end_idx].iterrows():
    # Skip if already successful
    if df.at[index, 'status'] == "Success":
        success_count += 1
        print(f"Row {index}: Already downloaded ✓")
        continue

    video_url = row['Video URL']
    start_time_str = row['Start time in the episode']
    end_time_str = row['End time in the episode']

    # DEBUG: Print raw values
    print(f"\n=== Row {index} DEBUG ===")
    print(f"Raw start: '{start_time_str}' | Raw end: '{end_time_str}'")

    start_seconds = timestamp_to_seconds(start_time_str)
    end_seconds = timestamp_to_seconds(end_time_str)

    # DEBUG: Check for None values
    if start_seconds is None or end_seconds is None:
        print(f"✗ Failed: Invalid timestamp conversion")
        print(f"  start_seconds={start_seconds}, end_seconds={end_seconds}")
        df.at[index, 'video_path'] = "DOWNLOAD_FAILED"
        df.at[index, 'status'] = "Invalid timestamp"
        failure_count += 1
        continue

    # Apply buffer
    buffer_start = max(0, start_seconds - START_BUFFER)
    buffer_end = end_seconds + END_BUFFER

    clip_num = success_count + failure_count + 1 - already_done
    total_to_process = end_idx - start_idx - already_done

    print(f"\n=== Row {index} ({clip_num}/{total_to_process}) ===")
    print(f"Timestamp: {start_time_str} to {end_time_str}")
    print(f"Seconds: {start_seconds}s to {end_seconds}s (duration: {end_seconds - start_seconds}s)")
    print(f"Buffer range: {buffer_start}s to {buffer_end}s")
    print(f"Video URL: {video_url[:60]}...")

    file_name = f"{index:04d}"
    full_output_path = os.path.join(video_directory, file_name)

    try:
        ydl_opts = {
            'format': 'best[height<=720]',
            'merge_output_format': 'mp4',
            'download_ranges': yt_dlp.utils.download_range_func(None, [(buffer_start, buffer_end)]),
            'force_keyframes_at_cuts': True,
            'noplaylist': True,
            'postprocessor_args': ['-c', 'copy'],
            'outtmpl': full_output_path,
            'cookiefile': cookie_path,
            'quiet': False,  # Changed to see errors
            'no_warnings': False,  # Changed to see warnings
            'verbose': True,  # Added for more debug info
        }

        print("Attempting download...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])

        # Find created file (handles any extension)
        possible_files = glob.glob(f"{full_output_path}*")

        if possible_files:
            output_file = possible_files[0]
            file_size = os.path.getsize(output_file) / 1024

            if file_size > 10:
                df.at[index, 'video_path'] = output_file
                df.at[index, 'status'] = "Success"
                success_count += 1
                print(f"✓ {os.path.basename(output_file)} ({file_size:.1f} KB)")
            else:
                raise Exception(f"File too small: {file_size:.1f} KB")
        else:
            raise Exception("No output file found")

    except Exception as e:
        error_msg = str(e)
        print(f"✗ Failed: {error_msg}")

        # Try to get more specific error info
        import traceback
        print("Full error traceback:")
        print(traceback.format_exc())

        df.at[index, 'video_path'] = "DOWNLOAD_FAILED"
        df.at[index, 'status'] = error_msg[:200]  # Truncate if too long
        failure_count += 1

    # Save progress after each download
    df.to_csv(output_csv_path, index=False)

    # Rate limiting
    time.sleep(0.5)

# -----------------------------
# BATCH SUMMARY
# -----------------------------
elapsed_time = time.time() - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"\n{'='*50}")
print(f"BATCH {START_INDEX}-{end_idx-1} COMPLETE")
print(f"{'='*50}")
print(f"✓ Success: {success_count}")
print(f"✗ Failed: {failure_count}")
print(f"⏱️  Time: {minutes}m {seconds}s")

if success_count + failure_count > 0:
    success_rate = success_count / (success_count + failure_count) * 100
    print(f"📊 Success rate: {success_rate:.1f}%")

    # Estimate remaining time
    if success_count + failure_count - already_done > 0:
        avg_time_per_clip = elapsed_time / (success_count + failure_count - already_done)
        remaining_clips = len(df) - end_idx
        remaining_batches = (remaining_clips + BATCH_SIZE - 1) // BATCH_SIZE
        estimated_minutes = (remaining_clips * avg_time_per_clip) / 60
        print(f"⏳ Estimated time for remaining {remaining_clips} clips: ~{estimated_minutes:.0f}m ({remaining_batches} batches)")

# Show dataset progress
total_success = df['status'].eq("Success").sum()
total_failed = (~df['status'].eq("Success") & ~df['status'].eq("")).sum()
print(f"\n📁 OVERALL DATASET PROGRESS")
print(f"Total downloaded: {total_success}/{len(df)} ({total_success/len(df)*100:.1f}%)")
print(f"Total failed: {total_failed}")

# Show next steps
if end_idx < len(df):
    print(f"\n🔄 TO CONTINUE:")
    print(f"Change START_INDEX to {end_idx}")
    print(f"Then run this cell again")
else:
    print(f"\n🎉 ALL CLIPS PROCESSED!")
    print(f"Check your CSV: {output_csv_path}")

[debug] Encodings: locale utf-8, fs utf-8, pref utf-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2025.11.12 from yt-dlp/yt-dlp [335653be8] (pip) API
[debug] params: {'format': 'best[height<=720]', 'merge_output_format': 'mp4', 'download_ranges': yt_dlp.utils.download_range_func(None, [(92100.0, 92340.0)]), 'force_keyframes_at_cuts': True, 'noplaylist': True, 'postprocessor_args': ['-c', 'copy'], 'outtmpl': '/content/drive/MyDrive/Final-Year-Project/data/WebScrapedVideos/0020', 'cookiefile': '/content/drive/MyDrive/Final-Year-Project/Notebooks/cookies.txt', 'quiet': False, 'no_warnings': False, 'verbose': True, 'js_runtimes': {'deno': {}}, 'remote_components': set(), 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us


=== BATCH PROCESSING ===
Processing rows: 20 to 20
Batch size: 1
Start buffer: 0.0s | End buffer: 0.0s
Strategy: Face consistency priority (no buffer)


=== Row 20 DEBUG ===
Raw start: '1 day, 1:35:00' | Raw end: '1 day, 1:39:00'

=== Row 20 (1/1) ===
Timestamp: 1 day, 1:35:00 to 1 day, 1:39:00
Seconds: 92100s to 92340s (duration: 240s)
Buffer range: 92100.0s to 92340.0s
Video URL: https://www.youtube.com/watch?v=UCPIBQDI0QM&list=PL5feTlxqLh...
Attempting download...
[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=UCPIBQDI0QM&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=15&ab_channel=bhministry
[youtube:tab] Downloading just the video UCPIBQDI0QM because of --no-playlist


[debug] [youtube] Found YouTube account cookies
[debug] [youtube] [pot] PO Token Providers: none
[debug] [youtube] [pot] PO Token Cache Providers: memory
[debug] [youtube] [pot] PO Token Cache Spec Providers: webpo
[debug] [youtube] [jsc] JS Challenge Providers: bun (unavailable), deno (unavailable), node (unavailable), quickjs (unavailable)


[youtube] Extracting URL: https://www.youtube.com/watch?v=UCPIBQDI0QM
[youtube] UCPIBQDI0QM: Downloading webpage
[youtube] UCPIBQDI0QM: Downloading tv downgraded player API JSON
[youtube] UCPIBQDI0QM: Downloading web safari player API JSON


[youtube] UCPIBQDI0QM: Downloading m3u8 information


...

[info] UCPIBQDI0QM: Downloading 1 format(s): 93
[info] UCPIBQDI0QM: Downloading 1 time ranges: 92100.0-92340.0


[debug] Invoking ffmpeg downloader on "https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1763594998/ei/lf4daZKXKruD6dsPqZe0qQQ/ip/35.204.43.169/id/5023c80500c8d103/itag/93/source/youtube/requiressl/yes/ratebypass/yes/pfa/1/sgoap/clen%3D28195515%3Bdur%3D1742.146%3Bgir%3Dyes%3Bitag%3D140%3Blmt%3D1755306097785615/sgovp/clen%3D56464365%3Bdur%3D1742.120%3Bgir%3Dyes%3Bitag%3D134%3Blmt%3D1755306111529627/rqh/1/hls_chunk_host/rr3---sn-5hnekn7l.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/cps/0/met/1763573397,/mh/Ht/mm/31,26/mn/sn-5hnekn7l,sn-a5meknzl/ms/au,onr/mv/u/mvi/3/pl/22/rms/au,au/gcr/nl/siu/1/bui/AdEuB5TYjJSMrYRWqfsg29CmUV0gUf2u7OH6ENGUUO4K3SGMHpT6Dmg3cilfGWElzkESiE3jjw/spc/6b0G_CuYza4JecylOjS5IylVflcI5tXXnVW55-gZPwj3QgdUkXBVnjEfXsQNVyWv2OGEoQ4/vprv/1/playlist_type/CLEAN/dover/11/txp/5309224/mt/1763571195/fvip/3/keepalive/yes/fexp/51355912,51552689,51565115,51565681,51580968/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,pfa,sgoap,sgovp,rqh,xpc,gcr,siu,bui,spc,vprv,pl

[download] Sleeping 6.00 seconds as required by the site...
[download] Destination: /content/drive/MyDrive/Final-Year-Project/data/WebScrapedVideos/0020


[debug] ffmpeg command line: ffmpeg -y -loglevel verbose -headers 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-us,en;q=0.5
Sec-Fetch-Mode: navigate
' -ss 92100.0 -t -90358.0 -i https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1763594998/ei/lf4daZKXKruD6dsPqZe0qQQ/ip/35.204.43.169/id/5023c80500c8d103/itag/93/source/youtube/requiressl/yes/ratebypass/yes/pfa/1/sgoap/clen%3D28195515%3Bdur%3D1742.146%3Bgir%3Dyes%3Bitag%3D140%3Blmt%3D1755306097785615/sgovp/clen%3D56464365%3Bdur%3D1742.120%3Bgir%3Dyes%3Bitag%3D134%3Blmt%3D1755306111529627/rqh/1/hls_chunk_host/rr3---sn-5hnekn7l.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/cps/0/met/1763573397,/mh/Ht/mm/31,26/mn/sn-5hnekn7l,sn-a5meknzl/ms/au,onr/mv/u/mvi/3/pl/22/rms/au,au/gcr/nl/siu/1/bui/AdEuB5TYjJSMrYRWqfsg29CmUV0gUf2u7OH6ENGUUO4K3SGMHpT6Dmg3cilfGWElzkESiE3jjw/spc/

✗ Failed: ERROR: ffmpeg exited with code 1
Full error traceback:
Traceback (most recent call last):
  File "/tmp/ipython-input-2289195128.py", line 86, in <cell line: 0>
    ydl.download([video_url])
  File "/usr/local/lib/python3.12/dist-packages/yt_dlp/YoutubeDL.py", line 3662, in download
    self.__download_wrapper(self.extract_info)(
  File "/usr/local/lib/python3.12/dist-packages/yt_dlp/YoutubeDL.py", line 3635, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/yt_dlp/YoutubeDL.py", line 1685, in extract_info
    return self.__extract_info(url, self.get_info_extractor(key), download, extra_info, process)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/yt_dlp/YoutubeDL.py", line 1696, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-pac